In [52]:
import pandas as pd

location = pd.read_csv(r'C:\Users\imran\Desktop\MyData\Python\Project\city.csv')
uses = pd.read_csv(r'C:\Users\imran\Desktop\MyData\Python\Project\consumption.csv')
tariff = pd.read_csv(r'C:\Users\imran\Desktop\MyData\Python\Project\tariff.csv')
df_uses = pd.read_csv(r'C:\Users\imran\Desktop\MyData\Python\Project\modified.csv')

location

,city,region,population
0,Mumbai,West,"12,442,373"
1,Delhi,North,"11,034,555"
2,Bangalore,South,"8,436,675"
3,Chennai,South,"7,088,000"
4,Kolkata,East,"4,486,679"
5,Pune,West,"3,124,458"


In [42]:
location['city'] = location['city'].str.title()

In [47]:
print(f"{10000000:,}")

10,000,000


In [57]:
def con(x):
    y =  f"{int(x):,}"
    return y
print(con('1000000'))

1,000,000


In [71]:
location['population'] = location['population'].str.replace(',','')
location['population'] = location['population'].apply(lambda x: f"{int(x):,}")
location['population']

0    12,442,373
1    11,034,555
2     8,436,675
3     7,088,000
4     4,486,679
5     3,124,458
Name: population, dtype: object

In [108]:
location['population'] = location['population'].str.replace(',','')
location['population'] = location['population'].apply(con)
location['population']

location.to_csv(r'C:\Users\imran\Desktop\MyData\Python\Project\city.csv', index = False)

In [82]:
print(location)

        city region  population
0     Mumbai   West  12,442,373
1      Delhi  North  11,034,555
2  Bangalore  South   8,436,675
3    Chennai  South   7,088,000
4    Kolkata   East   4,486,679
5       Pune   West   3,124,458


In [83]:
uses

,city,date,kwh_consumed,month
0,Mumbai,2022-01-01,5596.06,2022-01
1,Mumbai,2022-01-02,4834.08,2022-01
2,Mumbai,2022-01-03,4719.02,2022-01
3,Mumbai,2022-01-04,4719.04,2022-01
4,Mumbai,2022-01-05,6895.06,2022-01
...,...,...,...,...
2185,Pune,2022-12-27,4764.46,2022-12
2186,Pune,2022-12-28,2787.45,2022-12
2187,Pune,2022-12-29,3931.62,2022-12
2188,Pune,2022-12-30,2364.65,2022-12


In [94]:
uses['date'] = pd.to_datetime(uses['date'], format = '%Y,%m,%d')

uses['date'].dt.day_name()

0        Saturday
1          Sunday
2          Monday
3         Tuesday
4       Wednesday
          ...    
2185      Tuesday
2186    Wednesday
2187     Thursday
2188       Friday
2189     Saturday
Name: date, Length: 2190, dtype: object

In [157]:
uses_m = uses.merge(location,left_on = 'city', right_on='city', how = 'left')
uses_m['date'] = pd.to_datetime (uses_m['date'], format = '%Y-%m-%d')
uses_m['day'] = uses_m['date'].dt.day_name()
uses_m['month'] = uses_m['date'].dt.month_name()
uses_m['year'] = uses_m['date'].dt.year
uses_m['weekday'] = uses_m['date'].dt.weekday

uses_m['monyr'] = uses_m['date'].dt.strftime ('%Y-%m')
uses_m = uses_m.merge(tariff, left_on = ['city','monyr'], right_on=['city','month'], how = 'left')
uses_m= uses_m.drop(columns =['month_y',])
df_uses =uses_m
df_uses.to_csv(r'C:\Users\imran\Desktop\MyData\Python\Project\modified.csv')

In [49]:

df_uses['bill_amt'] = (df_uses['kwh_consumed']*df_uses['tariff_per_unit']).round(2)

df_uses.pivot_table(index= 'month_x', columns = 'region', values = ['kwh_consumed','bill_amt'],
                    aggfunc = {'kwh_consumed':'sum','bill_amt': 'mean'}, margins = True, margins_name= 'total')


df_uses

,Unnamed: 0,city,date,kwh_consumed,month_x,region,population,day,year,weekday,monyr,tariff_per_unit,bill_amt
0,0,Mumbai,2022-01-01,5596.06,January,West,"12,442,373",Saturday,2022,5,2022-01,6.79,37997.25
1,1,Mumbai,2022-01-02,4834.08,January,West,"12,442,373",Sunday,2022,6,2022-01,6.79,32823.40
2,2,Mumbai,2022-01-03,4719.02,January,West,"12,442,373",Monday,2022,0,2022-01,6.79,32042.15
3,3,Mumbai,2022-01-04,4719.04,January,West,"12,442,373",Tuesday,2022,1,2022-01,6.79,32042.28
4,4,Mumbai,2022-01-05,6895.06,January,West,"12,442,373",Wednesday,2022,2,2022-01,6.79,46817.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2185,2185,Pune,2022-12-27,4764.46,December,West,"3,124,458",Tuesday,2022,1,2022-12,5.79,27586.22
2186,2186,Pune,2022-12-28,2787.45,December,West,"3,124,458",Wednesday,2022,2,2022-12,5.79,16139.34
2187,2187,Pune,2022-12-29,3931.62,December,West,"3,124,458",Thursday,2022,3,2022-12,5.79,22764.08
2188,2188,Pune,2022-12-30,2364.65,December,West,"3,124,458",Friday,2022,4,2022-12,5.79,13691.32


In [38]:
df_uses.groupby(['city','month_x'])['kwh_consumed'].max()

city       month_x  
Bangalore  April        8122.02
           August       8107.48
           December     7488.90
           February     6788.87
           January      7991.60
                         ...   
Pune       March        7725.74
           May          7463.57
           November     6563.93
           October      7450.49
           September    8891.71
Name: kwh_consumed, Length: 72, dtype: float64

In [86]:
t_consumption = df_uses.pivot_table(index= 'city',values = 'kwh_consumed', aggfunc = ('sum'))

per_capita_consumption = t_consumption.merge(location, left_on = 'city', right_on = 'city', how = 'left').drop(columns = 'region')

per_capita_consumption['per_capita_consumption']=per_capita_consumption['kwh_consumed']/(per_capita_consumption['population'].replace (',','',regex= True).astype(int))*100

per_capita_consumption


#per_capita_consumption['kwh_consumed'] = (
 #   per_capita_consumption['kwh_consumed']/
  #  per_capita_consumption['population'].replace (',','',regex= True).astype(int))*100
#per_capita_consumption

#per_capita_consumption['kwh_consumed']/(per_capita_consumption['population'].replace (',','',regex= True).astype(int))*100





,city,kwh_consumed,population,per_capita_consumption
0,Bangalore,1827043.72,"8,436,675",21.655969
1,Chennai,1789114.52,"7,088,000",25.241458
2,Delhi,1848933.67,"11,034,555",16.755852
3,Kolkata,1800848.03,"4,486,679",40.137662
4,Mumbai,1820412.66,"12,442,373",14.630751
5,Pune,1818824.92,"3,124,458",58.212494
